In [3]:
# Vitaly 
import requests
from bs4 import BeautifulSoup

pages = [
    "https://www.immoweb.be/en/classified/10657599",
    "https://www.immoweb.be/en/classified/10655844",
    "https://www.immoweb.be/en/classified/10656608",
    "https://www.immoweb.be/en/classified/10654023",
    "https://www.immoweb.be/en/classified/10655537",
    "https://www.immoweb.be/en/classified/10647497",
    "https://www.immoweb.be/en/classified/10642679",
    "https://www.immoweb.be/en/classified/10615819",
    "https://www.immoweb.be/en/classified/10132944",
    "https://www.immoweb.be/en/classified/10648643",
    "https://www.immoweb.be/en/classified/10657189",
    "https://www.immoweb.be/en/classified/10649460",
    "https://www.immoweb.be/en/classified/10643043",
    "https://www.immoweb.be/en/classified/10649446"
]

immo_data = dict()
data_categories = []

print(len(pages))
for page in pages:
    temp = page.split("/")
    property_num = temp[-1]

    immo_data[property_num] = dict()

    req = requests.get(page)
    if req.status_code != 200:
        print(req.status_code)
        print(page)
    else:
        page_content = req.content
        #print(page_content)

        s = BeautifulSoup(page_content, "html.parser")

        ### to scrape from JS
        ### to svcrape Address from Button

        for tag in s.find_all("th"):
            dataType = tag.contents[0].strip()
            dataValue = tag.findNext("td").contents[0].strip()
            if dataValue == "":
                ### Scrape from children
                dataValue = "???"
        
            immo_data[property_num]["dataType"] = dataValue
            if dataType not in data_categories:
                data_categories.append(dataType)

data_categories.sort()
print(len(data_categories))
for cat in data_categories:
    print(cat)

14
404
https://www.immoweb.be/en/classified/10615819
97

Accessible for disabled people
Address
Agent's name
Air conditioning
Armored door
As built plan
Attic
Available as of
Available date
Basement
Basement surface
Bathrooms
Bedroom 1 surface
Bedroom 2 surface
Bedroom 3 surface
Bedroom 4 surface
Bedroom 5 surface
Bedrooms
Building condition
CO₂ emission
Cadastral income
Caretaker
Common water heater
Conformity certification for fuel tanks
Connection to sewer network
Construction year
Covered parking spaces
Dining room
Double glazing
Dressing room
Elevator
Energy class
External reference
Flat land
Flood zone type
Floor
Furnished
Garden
Garden orientation
Garden surface
Gas, water & electricity
Heat pump
Heating type
How many fireplaces?
Intercom
Internet
Isolated
Jacuzzi
Kitchen surface
Kitchen type
Land is facing street
Latest land use designation
Laundry room
Living area
Living room
Living room surface
Monthly charges
Neighbourhood or locality
Number of floors
Number of frontages
Obl